In [3]:
import copy
import os
from datetime import *
from tkinter import *
from tkinter import messagebox

In [4]:
path = os.getcwd()+'\\'

root=Tk()
root.title('Ввод показаний счетчиков')
root.geometry('410x310+300+300')
#root["bg"] = "green"

append_check = IntVar()

open_menu = Menu()
open_menu.add_command(label = 'Показания')
open_menu.add_command(label = 'Потребление')
open_menu.add_command(label = 'Расчет')

main_menu = Menu()
main_menu.add_cascade(label = 'Файл')
main_menu.add_cascade(label = 'Правка')
main_menu.add_cascade(label = 'Вид')
main_menu.add_cascade(label = 'Открыть', menu = open_menu)

today = datetime.date(datetime.today())
current_year = today.year
current_month = today.month
current_day = today.day

area = 44.8
months=['Января', 'Февраля', 'Марта', 'Апреля', 'Мая', 'Июня', 'Июля', 'Августа', 'Сентября', 'Октября', 'Ноября', 'Декабря']
services = ['В/О', 'ГАЗ', 'ГВС', 'ДМФН', 'КАПР', 'ОТОП', 'РФ', 'Э/Э', 'СИРЖП', 'ХВС', 'ЦВ']
permanent_index = [1, 3, 4, 5, 6, 8, 10]
varios_index = [2, 7, 9]
print_consum_index = [0, 2, 5, 7, 9, 11]
print_indications_index = [2, 7, 9, 11]

str1_1 = 'Сумма к оплате:'
str1_2 = str(0) + ' р.'
str2 = 'В том числе:'
str3_1 = 'Вода:'
str3_2 = str(0) + ' р.'
str3_3 = str(0) + ' %'
str4_1 = 'Свет:'
str4_2 = str(0) + ' р.'
str4_3 = str(0) + ' %'
str5_1 = 'ТСЖ:'
str5_2 = str(0) + ' р.'
str5_3 = str(0) + ' %'
str6_1 = 'Прочее:'
str6_2 = str(0) + ' р.'
str6_3 = str(0) + ' %'
str7_1 = 'Сэкономлено:'
str7_2 = str(0) + ' р.'
str7_3 = str(0) + ' %'

In [5]:
def prev_month(date):
    m = date.month - 1
    y = date.year
    if m<=0:
        m = 12 + m
        y = date.year - 1
    return m,y

In [6]:
def deltadays(date1, date2):
    date1=datetime.toordinal(date1)
    date2=datetime.toordinal(date2)
    delta=max(date1, date2)-min(date1, date2)
    return delta

In [7]:
def numtoword_month(a):
    return months[a - 1]

In [8]:
def ifint(a):
    if a%1==0:
        return round(a, 0)
    else:
        return round(a, 2)

In [9]:
def isfloat(value):
    try:
        float(value)
        return True
    except ValueError:
        return False

In [10]:
def lines_count(file_name):
    i = 0
    with open(path+file_name,'r') as file:
        for line in file:
            i+=1
    file.close()
    return(i)

In [11]:
def read_last_line_as_list(file_name):
    table = []
    with open(path+file_name,'r') as file:
        lines_counter = 0
        lastline = lines_count(file_name)
        for line in file:
            lines_counter+=1
            if lines_counter==lastline:
                s = line.split('\t')
                for i in range(len(s)):
                    if isfloat(s[i]) and float(s[i])%1==0:
                        table.append(int(float(s[i])))
                    elif isfloat(s[i]) and float(s[i])%1>0:
                        table.append(float(s[i]))
                    else:
                        table.append(s[i])
    file.close()
    return (table)

In [12]:
def append_to_file(file_name, list_name):
    with open(path+file_name, 'a') as file:
        file.write('\n')
        for i in range(len(list_name)):
            if i==0:
                file.write(str(list_name[i]))
            else:
                file.write('\t' + str(list_name[i]))
    file.close()

In [13]:
def checking_input(msg, last_indication, limit):
    Error=True
    while Error==True:
        try:
            indication=int(input(msg))
            Error=False
            if indication >= last_indication and indication < (last_indication + limit):
                Error=False
            else:
                int(msg)
            return indication
        except ValueError:
            print('Показание должно быть целым числом не меньше предыдущего')
            Error=True

In [14]:
def input_indications(last_indications):
    water_limit=20
    energy_limit=300
    new_indications = [0, 1, 'gvs', 1, area, 0.6048, area, 'e/e', area, 'hvs', area, 'date']
    print('Прошлые показания:')
    print('ГВС: ', last_indications[2], " куб. м")
    print('ХВС: ', last_indications[9], " куб. м")
    print('Э/Э: ', last_indications[7], " кВт")

    new_indications[2] = checking_input('Введите показание ГВС ', last_indications[2], water_limit)
    new_indications[9] = checking_input('Введите показание ХВС ', last_indications[9], water_limit)
    new_indications[7] = checking_input('Введите показание Э/Э ', last_indications[7], energy_limit)
    new_indications[11] = today
    return new_indications

In [15]:
def consumption(last_indications, new_indications):
    consumption = [0, 1, 0, 1, area, 0.6048, area, 0, area, 0, area]
    for i in varios_index:
        consumption[i] = new_indications[i] - last_indications[i]
    consumption[0] = consumption[2] + consumption[9]#VO
    print()
    print('{: <31}{: >8.0f}{: <}'.format('Потребление ГВС в этом месяце:', consumption[2], ' куб. м'))
    print('{: <31}{: >8.0f}{: <}'.format('Потребление ХВС в этом месяце:', consumption[9], ' куб. м'))
    print('{: <31}{: >8.0f}{: <}'.format('Водоотведение в этом месяце:', consumption[0], ' куб. м'))
    print('{: <31}{: >8.0f}{: <}'.format('Потребление Э/Э в этом месяце:', consumption[7], ' кВт'))
    consumption.append(today)
    return consumption

In [16]:
def cost(consumption, current_tariffs, preferences):
    rate = []
    economy = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    for i in range(len(consumption) - 1):
        if i==4:
            rate.append(consumption[i] * current_tariffs[i] - preferences[i])
            economy[4] = preferences[4]
        else:
            rate.append(consumption[i] * current_tariffs[i] * preferences[i])
            economy[i] = consumption[i] * current_tariffs[i] * (1 - preferences[i])
    total_cost = sum(rate)
    rate.append(total_cost)#rate[11]
    
    water_cost = rate[0] + rate[2] + rate[9]
    water_part = water_cost / total_cost
    rate.append(water_cost)#rate[12]

    light_part = rate[7] / total_cost
    rate.append(rate[7])#rate[13]
    
    tsg_cost = rate[6] + rate[8] + rate[10]
    tsg_part = tsg_cost / total_cost
    rate.append(tsg_cost)#rate[14]
    
    else_cost = rate[1] + rate[3] + rate[4] + rate[5]
    else_part = else_cost / total_cost
    else_cost = ifint(else_cost)
    rate.append(else_cost)#rate[15]
    total_economy = sum(economy)
    economy_part = total_economy / (total_economy + total_cost)
    rate.append(total_economy)#rate[16]

    for i in range(len(rate)):
        rate[i] = ifint(rate[i])
    ##    print()
##    print('{: <31}{: >8.2f}{: <}'.format('Сумма к оплате:', total_cost, ' р.'))
##    print()
##    print('В том числе:')
##    print('{: <31}{: >8.2f}{: <}{: >7.2%}'.format('Вода:', water_cost, ' р.', water_part))
##    print('{: <31}{: >8.2f}{: <}{: >7.2%}'.format('Свет:', rate[7], ' р.', light_part))
##    print('{: <31}{: >8.2f}{: <}{: >7.2%}'.format('ТСЖ:', tsg_cost, ' р.', tsg_part))
##    print('{: <31}{: >8.2f}{: <}{: >7.2%}'.format('Прочее:', else_cost, ' р.', else_part))
##    print()
##    print('{: <31}{: >8.2f}{: <}{: >7.2%}'.format('Сэкономлено:', total_economy, ' р.', economy_part))
    rate.append(today)#rate[17]

    str1_1 = 'Сумма к оплате:'
    str1_2 = '{:.2f}'.format(total_cost) + ' р.'
    str2 = 'В том числе:'
    str3_1 = 'Вода:'
    str3_2 = '{:.2f}'.format(water_cost) + ' р.'
    str3_3 = '{:.2%}'.format(water_part)
    str4_1 = 'Свет:'
    str4_2 = '{:.2f}'.format(rate[7]) + ' р.'
    str4_3 = '{:.2%}'.format(light_part)
    str5_1 = 'ТСЖ:'
    str5_2 = '{:.2f}'.format(tsg_cost) + ' р.'
    str5_3 = '{:.2%}'.format(tsg_part)
    str6_1 = 'Прочее:'
    str6_2 = '{:.2f}'.format(else_cost) + ' р.'
    str6_3 = '{:.2%}'.format(else_part)
    str7_1 = 'Сэкономлено:'
    str7_2 = '{:.2f}'.format(total_economy) + ' р.'
    str7_3 = '{:.2%}'.format(economy_part)

    l_str1_2.config(text = str1_2)
    l_str3_2.config(text = str3_2)
    l_str3_3.config(text = str3_3)
    l_str4_2.config(text = str4_2)
    l_str4_3.config(text = str4_3)
    l_str5_2.config(text = str5_2)
    l_str5_3.config(text = str5_3)
    l_str6_2.config(text = str6_2)
    l_str6_3.config(text = str6_3)
    l_str7_2.config(text = str7_2)
    l_str7_3.config(text = str7_3)
    return rate

In [17]:
def clear():
    ent_gvs.delete(0, END)
    ent_hvs.delete(0, END)
    ent_ee.delete(0, END)

In [18]:
def show():
    calc_permit = True
    water_limit = 20
    energy_limit = 300
    new_indications = [0, 1, 'gvs', 1, area, 0.6048, area, 'e/e', area, 'hvs', area, 'date']

    gvs_prev = last_indications[2]
    hvs_prev = last_indications[9]
    ee_prev = last_indications[7]

    try:
        new_indications[2] = int(ent_gvs.get())
    except:
            ent_gvs.delete(0, END)
            messagebox.showerror(title = 'Ошибка', message = 'Показание ГВС должно быть целым числом\nне меньше предыдущего')
            ent_gvs.focus()
            ent_gvs.config(bg = 'red')
    try:
        new_indications[9] = int(ent_hvs.get())
    except:
            ent_hvs.delete(0, END)
            messagebox.showerror(title = 'Ошибка', message = 'Показание ХВС должно быть целым числом\nне меньше предыдущего')
            ent_hvs.focus()
            ent_hvs.config(bg = 'red')
    try:
        new_indications[7] = int(ent_ee.get())
    except:
            ent_ee.delete(0, END)
            messagebox.showerror(title = 'Ошибка', message = 'Показание Э/Э должно быть целым числом\nне меньше предыдущего')
            ent_ee.focus()
            ent_ee.config(bg = 'red')
    new_indications[11] = today

    ent_gvs.config(bg = 'white')
    ent_hvs.config(bg = 'white')
    ent_ee.config(bg = 'white')

    if type(ent_gvs.get()) == float or new_indications[2] < gvs_prev:
            ent_gvs.delete(0, END)
            messagebox.showerror(title = 'Ошибка', message = 'Показание ГВС должно быть целым числом\nне меньше предыдущего')
            ent_gvs.focus()
            ent_gvs.config(bg = 'red')
            calc_permit = False

    if type(ent_hvs.get()) == float or new_indications[9] < hvs_prev:
            ent_hvs.delete(0, END)
            messagebox.showerror(title = 'Ошибка', message = 'Показание ХВС должно быть целым числом\nне меньше предыдущего')
            ent_hvs.focus()
            ent_hvs.config(bg = 'red')
            calc_permit = False

    if type(ent_ee.get()) == float or new_indications[7] < ee_prev:
            ent_ee.delete(0, END)
            messagebox.showerror(title = 'Ошибка', message = 'Показание Э/Э должно быть целым числом\nне меньше предыдущего')
            ent_ee.focus()
            ent_ee.config(bg = 'red')
            calc_permit = False

    if calc_permit:
        ent_gvs.config(bg = 'white')
        ent_hvs.config(bg = 'white')
        ent_ee.config(bg = 'white')
        consumption = [0, 1, 0, 1, area, 0.6048, area, 0, area, 0, area]
        for i in varios_index:
            consumption[i] = new_indications[i] - last_indications[i]
        consumption[0] = consumption[2] + consumption[9]#VO
        consumption.append(today)

        preferences = read_last_line_as_list('preferences.txt')

        rate = cost(consumption, current_tariffs, preferences)

        print_indications = []
        for i in print_indications_index:
            print_indications.append(new_indications[i])

        print_consum = []
        for i in print_consum_index:
            print_consum.append(consumption[i])

        rez = []
        print()
        rez.append('Показания квартирных счётчиков воды')
        rez.append('За период с 20 {:} {:} г. по 20 {:} {:} г.'.format(numtoword_month(previos_period_month), previos_period_year, numtoword_month(current_month), current_year))
        rez.append('ЖСК "Информатор"')
        rez.append('Ф.И.О.: Ивенский Никита Андреевич, тел. 8 (915) 094-93-50')
        rez.append('Адрес: Москва, ул. 2-я Пугачевская, д. 3, к. 1, кв. 85')
        rez.append('Дата заполнения: 20 {:} {:} г.'.format(numtoword_month(current_month), current_year))
        rez.append('{: <32} {: <31}'.format('Показания счётчика холодной воды', 'Показания счётчика горячей воды'))
        rez.append('{: <32} {: <31}'.format('Заводской №', 'Заводской №'))#AW80604454 - hvs, AW80907400 - gvs
        rez.append('{: <28}{: >4} {: <27}{: >4}'.format('Текущее, м3', new_indications[9], 'Текущее, м3', new_indications[2]))
        rez.append('{: <28}{: >4} {: <27}{: >4}'.format('Предыдущее, м3', prev_indications[2], 'Предыдущее, м3', prev_indications[0]))
        rez.append('{: <28}{: >4} {: <27}{: >4}'.format('Расход, м3', print_consum[4], 'Расход, м3', print_consum[1]))
        rez.append('')
        rez.append('Плательщик __________')

        if append_check.get() == 1:
            append_to_file('indications.txt', print_indications)
            append_to_file('consumption.txt', print_consum)
            append_to_file('cost.txt', rate)
            with open(path+'rez.txt','a') as file:
                for i in range(len(rez)):
                    print(rez[i])
                    file.write(str(rez[i]) + '\n')
                file.write('----------------------------------------------------------------\n')
            file.close()

        l_gvs_c.config(text = '{}'.format(consumption[2]))
        l_hvs_c.config(text = '{}'.format(consumption[9]))
        l_ee_c.config(text = '{}'.format(consumption[7]))

In [19]:
btn = Button(text = 'Ввести!', background = '#555', foreground = '#ccc', command = show)
btn.grid(row = 5, column = 1)
clear_btn = Button(text = 'Очистить', background = '#555', foreground = '#ccc', command = clear)
clear_btn.grid(row = 5, column = 2)

ent_gvs = Entry(justify = RIGHT)
ent_hvs = Entry(justify = RIGHT)
ent_ee = Entry(justify = RIGHT)
ent_gvs.grid(row = 1, column = 1)
ent_hvs.grid(row = 2, column = 1)
ent_ee.grid(row = 3, column = 1)

l_gvs_ct = Label(text = 'ГВС, куб. м')
l_hvs_ct = Label(text = 'ХВС, куб. м')
l_ee_ct = Label(text = 'Э/Э, кВт')
l_gvs_ct.grid(row = 1, column = 0)
l_hvs_ct.grid(row = 2, column = 0)
l_ee_ct.grid(row = 3, column = 0)

l_gvs_previn = Label(text = 100)
l_hvs_previn = Label(text = 200)
l_ee_previn = Label(text = 500)
l_gvs_previn.grid(row = 1, column = 2, sticky = E, padx='5', pady='1')
l_hvs_previn.grid(row = 2, column = 2, sticky = E, padx='5', pady='1')
l_ee_previn.grid(row = 3, column = 2, sticky = E, padx='5', pady='1')

l_gvs_c = Label(text = 0)
l_hvs_c = Label(text = 0)
l_ee_c = Label(text = 0)
l_gvs_c.grid(row = 1, column = 3, sticky = E, padx='5', pady='1')
l_hvs_c.grid(row = 2, column = 3, sticky = E, padx='5', pady='1')
l_ee_c.grid(row = 3, column = 3, sticky = E, padx='5', pady='1')

l_title = Label(text = 'Предыдущие\nпоказания')
l_curin = Label(text = 'Текущие\nпоказания')
l_cons = Label(text = 'Потребление')
l_curin.grid(row = 0, column = 1, padx='5', pady='5')
l_title.grid(row = 0, column = 2, padx='5', pady='5')
l_cons.grid(row = 0, column = 3, columnspan = 1, padx='5', pady='5')

l_str1_1 = Label(text = str1_1)
l_str1_2 = Label(text = str1_2)
l_str2 = Label(text = str2)
l_str3_1 = Label(text = str3_1)
l_str3_2 = Label(text = str3_2)
l_str3_3 = Label(text = str3_3)
l_str4_1 = Label(text = str4_1)
l_str4_2 = Label(text = str4_2)
l_str4_3 = Label(text = str4_3)
l_str5_1 = Label(text = str5_1)
l_str5_2 = Label(text = str5_2)
l_str5_3 = Label(text = str5_3)
l_str6_1 = Label(text = str6_1)
l_str6_2 = Label(text = str6_2)
l_str6_3 = Label(text = str6_3)
l_str7_1 = Label(text = str7_1)
l_str7_2 = Label(text = str7_2)
l_str7_3 = Label(text = str7_3)

l_str1_1.grid(row = 6, column = 0, columnspan = 1, sticky = W)
l_str1_2.grid(row = 6, column = 1, columnspan = 1, sticky = E)
l_str2.grid(row = 7, column = 0, columnspan = 5, sticky = W)
l_str3_1.grid(row = 8, column = 0, columnspan = 1, sticky = W)
l_str3_2.grid(row = 8, column = 1, columnspan = 1, sticky = E)
l_str3_3.grid(row = 8, column = 2, columnspan = 1, sticky = E)
l_str4_1.grid(row = 9, column = 0, columnspan = 1, sticky = W)
l_str4_2.grid(row = 9, column = 1, columnspan = 1, sticky = E)
l_str4_3.grid(row = 9, column = 2, columnspan = 1, sticky = E)
l_str5_1.grid(row = 10, column = 0, columnspan = 1, sticky = W)
l_str5_2.grid(row = 10, column = 1, columnspan = 1, sticky = E)
l_str5_3.grid(row = 10, column = 2, columnspan = 1, sticky = E)
l_str6_1.grid(row = 11, column = 0, columnspan = 1, sticky = W)
l_str6_2.grid(row = 11, column = 1, columnspan = 1, sticky = E)
l_str6_3.grid(row = 11, column = 2, columnspan = 1, sticky = E)
l_str7_1.grid(row = 12, column = 0, columnspan = 1, sticky = W)
l_str7_2.grid(row = 12, column = 1, columnspan = 1, sticky = E)
l_str7_3.grid(row = 12, column = 2, columnspan = 1, sticky = E)

append_checkbutton = Checkbutton(text = 'Запись в файлы', variable = append_check)
append_checkbutton.grid(row = 4, column = 1, sticky = W)

In [20]:
previos_period_month, previos_period_year=prev_month(today)    
current_tariffs = read_last_line_as_list('tariffs.txt')
prev_indications = read_last_line_as_list('indications.txt')
last_indications = [0, 1, prev_indications[0], 1, area, 0.6048, area, prev_indications[1], area, prev_indications[2], area, prev_indications[3]]

In [21]:
l_gvs_previn.config(text = last_indications[2])
l_hvs_previn.config(text = last_indications[9])
l_ee_previn.config(text = last_indications[7])

In [ ]:
ent_gvs.focus()
root.config(menu = main_menu)
root.mainloop()

In [ ]:
prev_date_string=prev_indications[3]
prev_date=datetime.strptime(prev_date_string,'%Y-%m-%d')
prev_date=datetime.date(prev_date)
delta_days=deltadays(today, prev_date)